任何策略的本质都是在预测，一次预测包括用来预测的因子、被预测的变量、预测模型。预测是一个研究过程，包括不断调整
预测因子和预测模型，以获得最小的错误率

下面选定分钟bar的涨跌作为被预测变量，选定非常细粒度的数据的好处在于，可以快速的累积到足够多的样本数据。 缺点在于，越是高粒度
的数据，其规律越接近于随机游走，所以发现有效模型的概率越低。 关于预测因子，在高频交易的情况下，由于不需要考虑基本面的数据，所以
预测因子基本上也是过去的回报。为了预测一分钟的回报方向，选定过去1、3、5分钟的回报作为预测因子，模型上选用逻辑回归模型。预测因子
可以尝试下连续型和离散型

In [2]:
from se2.domain.time_series import *
from se2.domain.engine import *
from se2.domain.account import *
from se2.domain.common import *
import pandas as pd
import numpy as np
import scipy.stats

In [3]:
code = 'SPCE_STK_USD_SMART'
start = pd.Timestamp('2021-05-09 21:00:00', tz='Asia/Shanghai')
end = pd.Timestamp('2021-06-09 21:00:00', tz='Asia/Shanghai')
command = HistoryDataQueryCommand(start, end, [code])
ts_repo: TimeSeriesRepo = BeanContainer.getBean(TimeSeriesRepo)
ts: TimeSeries = ts_repo.find_one("ibMinBar")
MIN_BAR_DF: DataFrame = ts.history_data(command, from_local=True)
MIN_BAR_DF = MIN_BAR_DF.droplevel(level=1)
MIN_BAR_DF.head()

,start_time,open,high,low,close,volume
visible_time,,,,,,
2021-05-10 21:31:00+08:00,2021-05-10 21:30:00+08:00,19.58,19.59,19.32,19.41,1624
2021-05-10 21:32:00+08:00,2021-05-10 21:31:00+08:00,19.39,19.39,19.12,19.16,387
2021-05-10 21:33:00+08:00,2021-05-10 21:32:00+08:00,19.15,19.35,19.07,19.30,432
2021-05-10 21:34:00+08:00,2021-05-10 21:33:00+08:00,19.30,19.45,19.29,19.41,299
2021-05-10 21:35:00+08:00,2021-05-10 21:34:00+08:00,19.37,19.38,19.27,19.27,190


In [36]:
# 构造训练集和验证集
df = MIN_BAR_DF[:'2021-05-11 21:00:00']
pre_ret1 = np.log(df['close'] / df['close'].shift(1))
pre_ret3 = np.log(df['close'] / df['close'].shift(3))
pre_ret5 = np.log(df['close'] / df['close'].shift(5))
next_ret1 = np.log(df['close'].shift(-1) / df['close'])
df = pd.DataFrame({"pre_ret1": pre_ret1, "pre_ret3": pre_ret3, "pre_ret5":pre_ret5, "next_ret1": next_ret1})
train_data = df.dropna()
train_data = train_data[train_data['next_ret1']!=0]
train_data.loc[train_data['next_ret1']>0, 'next_ret1'] = 1
train_data.loc[train_data['next_ret1']<0, 'next_ret1'] = -1

df = MIN_BAR_DF['2021-05-11 21:00:00':'2021-05-12 21:00:00']
pre_ret1 = np.log(df['close'] / df['close'].shift(1))
pre_ret3 = np.log(df['close'] / df['close'].shift(3))
pre_ret5 = np.log(df['close'] / df['close'].shift(5))
next_ret1 = np.log(df['close'].shift(-1) / df['close'])
df = pd.DataFrame({"pre_ret1": pre_ret1, "pre_ret3": pre_ret3, "pre_ret5":pre_ret5, "next_ret1": next_ret1})
test_data = df.dropna()
test_data = test_data[test_data['next_ret1']!=0]
test_data.loc[test_data['next_ret1']>0, 'next_ret1'] = 1
test_data.loc[test_data['next_ret1']<0, 'next_ret1'] = -1


In [42]:
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
X, y = load_iris(return_X_y=True)
clf = LogisticRegression(random_state=0).fit(X, y)
clf.predict(X[:2, :])

/Users/zhang/opt/miniconda3/envs/strategy-engine/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


array([0, 0])

In [40]:
X[:2, :]

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2]])

In [43]:
clf.predict_proba(X[:2, :])

array([[9.81802911e-01, 1.81970751e-02, 1.43580537e-08],
       [9.71729527e-01, 2.82704429e-02, 3.00353141e-08]])

In [44]:
clf.score(X, y)



0.9733333333333334

In [49]:
X = train_data[['pre_ret1','pre_ret3', 'pre_ret5']].values
y = train_data['next_ret1'].values
lr = LogisticRegression(random_state=0)
lr.fit(X, y)
X_t = test_data[['pre_ret1','pre_ret3', 'pre_ret5']].values
y_t = test_data['next_ret1'].values
lr.score(X_t, y_t)


0.45938375350140054

In [56]:
pd.concat([train_data['pre_ret5'], test_data['pre_ret5']]).describe()


count    696.000000
mean       0.000310
std        0.007099
min       -0.030332
25%       -0.003239
50%        0.000000
75%        0.003449
max        0.044397
Name: pre_ret5, dtype: float64

In [85]:
# 尝试使用离散的变量
new_train_data = train_data.copy()
new_train_data.loc[:,'pre_ret1'] = pd.qcut(new_train_data['pre_ret1'], 2, labels=[-1, 1])
new_train_data.loc[:,'pre_ret3'] = pd.qcut(new_train_data['pre_ret3'], 2, labels=[-1, 1])
new_train_data.loc[:,'pre_ret5'] = pd.qcut(new_train_data['pre_ret5'], 2, labels=[-1, 1])

new_test_data = test_data.copy()
new_test_data.loc[:,'pre_ret1'] = pd.qcut(new_test_data['pre_ret1'], 2, labels=[-1, 1])
new_test_data.loc[:,'pre_ret3'] = pd.qcut(new_test_data['pre_ret3'], 2, labels=[-1, 1])
new_test_data.loc[:,'pre_ret5'] = pd.qcut(new_test_data['pre_ret5'], 2, labels=[-1, 1])


In [86]:
X = new_train_data[['pre_ret1','pre_ret3', 'pre_ret5']].values
y = new_train_data['next_ret1'].values
lr = LogisticRegression(random_state=0)
lr.fit(X, y)
X_t = new_test_data[['pre_ret1','pre_ret3', 'pre_ret5']].values
y_t = new_test_data['next_ret1'].values
lr.score(X_t, y_t)

0.5210084033613446

In [95]:
MIN_BAR_DF[:'2021-05-11 21:00:00']

,start_time,open,high,low,close,volume
visible_time,,,,,,
2021-05-10 21:31:00+08:00,2021-05-10 21:30:00+08:00,19.58,19.59,19.32,19.41,1624
2021-05-10 21:32:00+08:00,2021-05-10 21:31:00+08:00,19.39,19.39,19.12,19.16,387
2021-05-10 21:33:00+08:00,2021-05-10 21:32:00+08:00,19.15,19.35,19.07,19.30,432
2021-05-10 21:34:00+08:00,2021-05-10 21:33:00+08:00,19.30,19.45,19.29,19.41,299
2021-05-10 21:35:00+08:00,2021-05-10 21:34:00+08:00,19.37,19.38,19.27,19.27,190
...,...,...,...,...,...,...
2021-05-11 03:56:00+08:00,2021-05-11 03:55:00+08:00,17.92,17.97,17.91,17.96,875
2021-05-11 03:57:00+08:00,2021-05-11 03:56:00+08:00,17.97,17.98,17.97,17.98,481
2021-05-11 03:58:00+08:00,2021-05-11 03:57:00+08:00,17.98,17.98,17.90,17.91,1459


In [116]:
from sklearn.svm import SVC

discretization_count = 2
labels = [-1, 1]

df = MIN_BAR_DF[:'2021-05-11 21:00:00']
pre_ret1 = np.log(df['close'] / df['close'].shift(1))
pre_ret3 = np.log(df['close'] / df['close'].shift(3))
pre_ret5 = np.log(df['close'] / df['close'].shift(5))
pre_ret10 = np.log(df['close'] / df['close'].shift(10))
pre_volume1 = df['volume']
next_ret1 = np.log(df['close'].shift(-1) / df['close'])
df = pd.DataFrame({"pre_ret1": pre_ret1, "pre_ret3": pre_ret3, "pre_ret5":pre_ret5,
                   "pre_volume1": pre_volume1,
                   "pre_ret10": pre_ret10,
                   "next_ret1": next_ret1})
train_data = df.dropna()
train_data = train_data[train_data['next_ret1']!=0]
train_data.loc[train_data['next_ret1']>0, 'next_ret1'] = 1
train_data.loc[train_data['next_ret1']<0, 'next_ret1'] = -1

df = MIN_BAR_DF['2021-05-11 21:00:00':'2021-05-12 21:00:00']
pre_ret1 = np.log(df['close'] / df['close'].shift(1))
pre_ret3 = np.log(df['close'] / df['close'].shift(3))
pre_ret5 = np.log(df['close'] / df['close'].shift(5))
pre_ret10 = np.log(df['close'] / df['close'].shift(10))
pre_volume1 = df['volume']
next_ret1 = np.log(df['close'].shift(-1) / df['close'])
df = pd.DataFrame({"pre_ret1": pre_ret1, "pre_ret3": pre_ret3, "pre_ret5":pre_ret5,
                   "pre_volume1": pre_volume1,
                   "pre_ret10": pre_ret10,
                   "next_ret1": next_ret1})
test_data = df.dropna()
test_data = test_data[test_data['next_ret1']!=0]
test_data.loc[test_data['next_ret1']>0, 'next_ret1'] = 1
test_data.loc[test_data['next_ret1']<0, 'next_ret1'] = -1


new_train_data = train_data.copy()
new_train_data.loc[:,'pre_ret1'] = pd.qcut(new_train_data['pre_ret1'], discretization_count, labels=labels)
new_train_data.loc[:,'pre_ret3'] = pd.qcut(new_train_data['pre_ret3'], discretization_count, labels=labels)
new_train_data.loc[:,'pre_ret5'] = pd.qcut(new_train_data['pre_ret5'], discretization_count, labels=labels)
new_train_data.loc[:,'pre_volume1'] = pd.qcut(new_train_data['pre_volume1'], discretization_count, labels=labels)
new_train_data.loc[:,'pre_ret10'] = pd.qcut(new_train_data['pre_ret10'], discretization_count, labels=labels)

new_test_data = test_data.copy()
new_test_data.loc[:,'pre_ret1'] = pd.qcut(new_test_data['pre_ret1'], discretization_count, labels=labels)
new_test_data.loc[:,'pre_ret3'] = pd.qcut(new_test_data['pre_ret3'], discretization_count, labels=labels)
new_test_data.loc[:,'pre_ret5'] = pd.qcut(new_test_data['pre_ret5'], discretization_count, labels=labels)
new_test_data.loc[:,'pre_volume1'] = pd.qcut(new_test_data['pre_volume1'], discretization_count, labels=labels)
new_test_data.loc[:,'pre_ret10'] = pd.qcut(new_test_data['pre_ret10'], discretization_count, labels=labels)


X = new_train_data[['pre_ret1','pre_ret3', 'pre_ret5']].values
y = new_train_data['next_ret1'].values
model = SVC()
model.fit(X, y)
print("train data score with svc:{}".format(model.score(X, y)))
model2 = LogisticRegression()
model2.fit(X, y)
print("train data score with lr:{}".format(model2.score(X, y)))
X_t = new_test_data[['pre_ret1','pre_ret3', 'pre_ret5']].values
y_t = new_test_data['next_ret1'].values
print("test data score with svc:{}".format(model.score(X_t, y_t)))
print("test data score with lr:{}".format(model2.score(X_t, y_t)))

train data score with svc:0.5688622754491018
train data score with lr:0.562874251497006
test data score with svc:0.547752808988764
test data score with lr:0.5224719101123596


In [117]:
# 从上面的实验结果，在不对模型进行进一步调优的情况下，使用预测分钟的涨跌没有表现出显著性。显著性
# 的检验使用二项分布，在n次独立重复实验中恰好出现k次的概率是(n!/k!(n-k)!)p^k(1-p)^k。
# 下面考虑时间粒度更大的数据的预测


,start_time,open,high,low,close,volume
visible_time,,,,,,
2021-05-10 21:31:00+08:00,2021-05-10 21:30:00+08:00,19.58,19.59,19.32,19.41,1624
2021-05-10 21:32:00+08:00,2021-05-10 21:31:00+08:00,19.39,19.39,19.12,19.16,387
2021-05-10 21:33:00+08:00,2021-05-10 21:32:00+08:00,19.15,19.35,19.07,19.30,432
2021-05-10 21:34:00+08:00,2021-05-10 21:33:00+08:00,19.30,19.45,19.29,19.41,299
2021-05-10 21:35:00+08:00,2021-05-10 21:34:00+08:00,19.37,19.38,19.27,19.27,190
...,...,...,...,...,...,...
2021-06-09 03:56:00+08:00,2021-06-09 03:55:00+08:00,37.50,37.54,37.44,37.45,720
2021-06-09 03:57:00+08:00,2021-06-09 03:56:00+08:00,37.44,37.57,37.42,37.52,777
2021-06-09 03:58:00+08:00,2021-06-09 03:57:00+08:00,37.53,37.58,37.49,37.53,893


In [157]:
PRED = [1, 5, 10, 20, 30]
N = 10
# 按天计算预测因子（过去的不同时间粒度回报）和被预测变量（未来一段时间的回报）

def calc_ret(daily_df: DataFrame):
    if len(daily_df)<=0:
        return None
    ret_map = {}
    for k in PRED:
        ret_map["pre_ret"+str(k)] = np.log(daily_df['close'] / daily_df['close'].shift(k))
        ret_map["std_pre_ret"+str(k)] = np.power(np.log(daily_df['close'] / daily_df['close'].shift(k)),2)
    ret_map['next_ret'+str(N)] = np.log(daily_df['close'].shift(-N) / daily_df['close'])
    df = pd.DataFrame(ret_map).dropna()
    # 去掉未来回报为0的情况,因为被预测变量是二元离散变量
    df = df[df['next_ret'+str(N)]!=0]
    return df

rets = MIN_BAR_DF.groupby(pd.Grouper(freq='1D',origin=pd.Timestamp("21:00:00", tz='Asia/Shanghai'))).apply(calc_ret).dropna()
rets = rets.droplevel(level=0)
rets.head()

,pre_ret1,std_pre_ret1,pre_ret5,std_pre_ret5,pre_ret10,std_pre_ret10,pre_ret20,std_pre_ret20,pre_ret30,std_pre_ret30,next_ret10
visible_time,,,,,,,,,,,
2021-05-10 22:01:00+08:00,-0.005405,0.000029,-0.008097,0.000066,-0.018792,0.000353,-0.023568,0.000555,-0.050724,0.002573,0.001083
2021-05-10 22:02:00+08:00,0.003247,0.000011,-0.009142,0.000084,-0.013416,0.000180,-0.022437,0.000503,-0.034514,0.001191,0.009142
2021-05-10 22:03:00+08:00,-0.002705,0.000007,-0.007018,0.000049,-0.015054,0.000227,-0.026198,0.000686,-0.044499,0.001980,0.013986
2021-05-10 22:04:00+08:00,-0.002712,0.000007,-0.008653,0.000075,-0.012416,0.000154,-0.028910,0.000836,-0.052894,0.002798,0.013488
2021-05-10 22:05:00+08:00,0.005417,0.000029,-0.002159,0.000005,-0.005925,0.000035,-0.024020,0.000577,-0.040238,0.001619,0.008607


In [158]:
# 因子数据的离散化配置， 不同的离散化方式会对结果产生影响
# 下面尝试三种不同的离散化方式
factor_disc_config = {
    "2_0":{
        "bins":[-100, 0, 100],
        'labels':[-1, 1]
    },
    "2_q":{
        "qc":2,
        'labels':[-1, 1]
    },
    "4_q":{
        "qc":4,
        'labels':[-2, -1, 1, 2]
    }
}
next_ret_key = 'next_ret'+str(N)
rets.loc[rets[next_ret_key]>0, next_ret_key+"_disc"] = 1
rets.loc[rets[next_ret_key]<0, next_ret_key+"_disc"] = -1
for k in PRED:
    for disc_key in factor_disc_config.keys():
        key = 'pre_ret'+str(k)
        new_key = "pre_ret"+str(k)+"_disc"+disc_key
        conf = factor_disc_config[disc_key]
        if 'bins' in conf:
            rets.loc[:, new_key] = pd.cut(rets[key], conf['bins'], labels=conf['labels'])
        else:
            rets.loc[:, new_key] = pd.qcut(rets[key], conf['qc'], labels=conf['labels'])

        key = 'std_pre_ret'+str(k)
        new_key = "std_pre_ret"+str(k)+"_disc"+disc_key
        conf = factor_disc_config[disc_key]
        if 'bins' in conf:
            rets.loc[:, new_key] = pd.cut(rets[key], conf['bins'], labels=conf['labels'])
        else:
            rets.loc[:, new_key] = pd.qcut(rets[key], conf['qc'], labels=conf['labels'])

In [159]:
# 划分训练集和测试集
split_index = int(len(rets) / 2)
TRAIN_DATA = rets.iloc[ :split_index]
TEST_DATA = rets.iloc[split_index: ]

In [173]:
# 模型适配
# 为了方便对比，实验次数应该是模型数量跟因子的笛卡尔积
tests_confg = {
    # "all":["pre_ret1", "pre_ret5", "pre_ret10", "pre_ret20"],
    # "all_disc2_0":["pre_ret1_disc2_0", "pre_ret5_disc2_0", 'pre_ret10_disc2_0', 'pre_ret20_disc2_0'],
    # "all_disc2_q":["pre_ret1_disc2_q", "pre_ret5_disc2_q", 'pre_ret10_disc2_q', 'pre_ret20_disc2_q'],
    # "all_disc4_q":["pre_ret1_disc4_q", "pre_ret5_disc4_q", 'pre_ret10_disc4_q', 'pre_ret20_disc4_q']
    "all":["pre_ret5",'pre_ret10'],
    "all_disc2_0":["pre_ret5_disc2_0", "pre_ret10_disc2_0"],
    # "all_disc4_q":["pre_ret5_disc4_q", "pre_ret10_disc4_q", 'pre_ret20_disc4_q', 'pre_ret30_disc4_q'],
    # "all_with_std":["pre_ret5", "pre_ret10", "std_pre_ret5", "std_pre_ret10"],
    # "all_with_std2_q":["pre_ret5_disc2_q", "pre_ret10_disc2_q", "std_pre_ret5_disc2_q", "std_pre_ret10_disc2_q"],
    # "all_with_std2_q":["pre_ret5_disc2_0", "pre_ret10_disc2_0", "std_pre_ret5_disc2_0", "std_pre_ret10_disc2_0"],
    # "all_disc2_0":["pre_ret5_disc2_0", 'pre_ret10_disc2_0'],
    # "all_disc2_q":["pre_ret5_disc2_q", 'pre_ret10_disc2_q'],
    # "all_disc4_q":["pre_ret5_disc4_q", 'pre_ret10_disc4_q']
}
predicted_key = 'next_ret10_disc'

res = {}
for factor_key in tests_confg.keys():
    scores = {}
    factors = tests_confg[factor_key]
    X = TRAIN_DATA[factors].values
    y = TRAIN_DATA[predicted_key].values
    model = SVC()
    model.fit(X, y)
    scores['svc_train_data'] = model.score(X, y)
    model2 = LogisticRegression()
    model2.fit(X, y)
    scores['lr_train_data'] = model2.score(X, y)
    X_t = TEST_DATA[factors].values
    y_t = TEST_DATA[predicted_key].values
    scores['svc_test_data'] = model.score(X_t, y_t)
    scores['lr_test_data'] = model2.score(X_t, y_t)

    res[factor_key] = scores

res

{'all': {'svc_train_data': 0.5518497757847534,
  'lr_train_data': 0.5134529147982063,
  'svc_test_data': 0.5381165919282511,
  'lr_test_data': 0.5302690582959642},
 'all_disc2_0': {'svc_train_data': 0.5291479820627802,
  'lr_train_data': 0.5252242152466368,
  'svc_test_data': 0.5196188340807175,
  'lr_test_data': 0.5109304932735426}}

In [ ]:
# 从上面的实验看来，无论是使用离散的预测因子，还是连续的预测因子。无论是使用SVM模型，还是
# 逻辑回归模型。 无论是用多长时间区间的历史回报数据来预测未来任意时间区间的回报数据（实验尝试了对未来1分钟、5分钟、10分钟的
# 的回报方向进行预测，结果都是不显著的）。猜想原因是因为因子维度太单一导致的
# 使用连续因子的结果会略微好于离散因子，也许是因为连续因子能够提供更多的信息，这些信息
# 减少的不确定性多余增加的噪音。